# Telecom Churn Group Project

This notebook is split into following section

<b>* Common Function</b>
    * This section we have all the common functions which are used in the entire notebook
<b> * Basic Data Analysis and Null Value imputation </b>
    * Drop Columns with more than 50% NULL Values
    * Handle columns with less number of NULL Values
    * Drop columns having not informative information
    * Create Dummy Variables for the categorical variables.
    * Filter and get high valued customer information
    * Derived Columns based on basic column analysis
    * Create a new columns which will tell about the Churn/No-Churn customer and Drop 9th month related columns
<b> * EDA </b>
    * EDA for Month 6 and 7 Together
    * EDA for Month 8
    * Derived Columns by combining 6th, 7th and 8th columns 
    * How the features are varied from Good Period to Decision period
    * What is the average variation from 6th+7th Month to 8th Month
    * EDA for the derived columns
<b> * Data Modeling </b>
    * Data Normalization 
    * Basic Logistic Regression Fit to Check the accuracy
    * As churn count is less, basic logist can be done with GridSearch and K-Fold
    * Using PCA dimentionality reduction can be done
    * By taking the PCA data logistic regression can be done again with GridSearch and K-Fold to check for prediction.
    * Using Ridge Regression, It can be found the important features impacting the churn.
    * Using Tree Model, Also Important Features can be derived.
    * If during EDA, any variable relation found with multinomial relation, then SVM Kernel can be used for prediction.

<b> * Final Model Selection </b>
    * Final Model for Prediction
    * Final Model for important Feature selection 
<b> * Summary </b>
    * Project Analysis and Summary

In [315]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import time
import datetime

df = pd.read_csv('telecom_churn_data.csv')

# Common Function

In [349]:
# Function to Return Montwise ColumnsList
def returnColumnsByMonth(df):
    column_Month_6 = []
    column_Month_7 = []
    column_Month_8 = []
    column_Month_9 = []
    column_Common = []
    for eachColumns in df.columns:
        if((eachColumns.find("_6") >=0) | (eachColumns.find("jun_") >=0)):
            column_Month_6.append(eachColumns)
        elif((eachColumns.find("_7") >=0) | (eachColumns.find("jul_") >=0)):
            column_Month_7.append(eachColumns)
        elif((eachColumns.find("_8") >= 0) | (eachColumns.find("aug_") >=0)):
            column_Month_8.append(eachColumns)
        elif((eachColumns.find("_9") >=0) | (eachColumns.find("sep_") >=0)):
            column_Month_9.append(eachColumns)
        else:
            column_Common.append(eachColumns)
    return column_Month_6, column_Month_7, column_Month_8, column_Month_9, column_Common

# Function to Get Columns Based on Null %
def getColumnsBasedOnNullPercent(df, nullPercentLimit, limitType = 'Upper'):
    col2NullPercent_df = pd.DataFrame(round((df.isnull().sum()/len(df.index))* 100, 2), columns=['NullPercent'])
    col2NullPercent_df = pd.DataFrame(round((df.isnull().sum()/len(df.index))* 100, 2), columns=['NullPercent'])
    if(limitType == 'Upper'):
        columnsList = np.array(col2NullPercent_df.apply(lambda x: x['NullPercent'] > nullPercentLimit , axis=1))
    if(limitType == 'Lower'):
        columnsList = np.array(col2NullPercent_df.apply(lambda x: ((x['NullPercent'] < nullPercentLimit) & (x['NullPercent'] > 0)) , axis=1))
    return np.array(df.loc[:, columnsList].columns)

def daysFromLastRechargeInMonth(lastRechargeDate, lastDateOfMonth):
    date_format = "%m/%d/%Y"
    delta = 30
    if(len(lastRechargeDate) > 0):
        a = datetime.strptime(lastRechargeDate, date_format)
        b = datetime.strptime(lastDateOfMonth, date_format)
    return delta

# Basic Data Analysis and Null Value Imputation

In [304]:
df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [305]:
df.columns

Index(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
       'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7',
       'last_date_of_month_8', 'last_date_of_month_9', 'arpu_6',
       ...
       'sachet_3g_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g'],
      dtype='object', length=226)

####  * Get Columns Monthwise & Basic Understanding of Columns

In [306]:
column_Month_6, column_Month_7, column_Month_8, column_Month_9, column_Common = returnColumnsByMonth(df)

print("Month 6 Columns Count ==> {}".format(len(column_Month_6)))
print("Month 7 Columns Count ==> {}".format(len(column_Month_7)))
print("Month 8 Columns Count ==> {}".format(len(column_Month_8)))
print("Month 9 Columns Count ==> {}".format(len(column_Month_9)))
print("Common Columns Count ==> {}".format(len(column_Common)))

Month 6 Columns Count ==> 55
Month 7 Columns Count ==> 55
Month 8 Columns Count ==> 55
Month 9 Columns Count ==> 55
Common Columns Count ==> 6


In [307]:
# All Months are having same type of columns So lets see the columns in general
print ("\nMonth based Columns:\n \t\t==> {}".format(np.array(column_Month_6)))
print ("\nCommon Columns:\n \t\t==> {}".format(np.array(column_Common)))


Month based Columns:
 		==> ['last_date_of_month_6' 'arpu_6' 'onnet_mou_6' 'offnet_mou_6'
 'roam_ic_mou_6' 'roam_og_mou_6' 'loc_og_t2t_mou_6' 'loc_og_t2m_mou_6'
 'loc_og_t2f_mou_6' 'loc_og_t2c_mou_6' 'loc_og_mou_6' 'std_og_t2t_mou_6'
 'std_og_t2m_mou_6' 'std_og_t2f_mou_6' 'std_og_t2c_mou_6' 'std_og_mou_6'
 'isd_og_mou_6' 'spl_og_mou_6' 'og_others_6' 'total_og_mou_6'
 'loc_ic_t2t_mou_6' 'loc_ic_t2m_mou_6' 'loc_ic_t2f_mou_6' 'loc_ic_mou_6'
 'std_ic_t2t_mou_6' 'std_ic_t2m_mou_6' 'std_ic_t2f_mou_6'
 'std_ic_t2o_mou_6' 'std_ic_mou_6' 'total_ic_mou_6' 'spl_ic_mou_6'
 'isd_ic_mou_6' 'ic_others_6' 'total_rech_num_6' 'total_rech_amt_6'
 'max_rech_amt_6' 'date_of_last_rech_6' 'last_day_rch_amt_6'
 'date_of_last_rech_data_6' 'total_rech_data_6' 'max_rech_data_6'
 'count_rech_2g_6' 'count_rech_3g_6' 'av_rech_amt_data_6' 'vol_2g_mb_6'
 'vol_3g_mb_6' 'arpu_3g_6' 'arpu_2g_6' 'night_pck_user_6' 'monthly_2g_6'
 'sachet_2g_6' 'monthly_3g_6' 'sachet_3g_6' 'fb_user_6' 'jun_vbc_3g']

Common Columns:
 		==

#### * Derive Columns Total_Recharge_Amount from 6th and 7th Month total_rech_amt

In [308]:
df['Total_Recharge_Amount'] = df['total_rech_amt_6'] + df['total_rech_amt_7']

# Get 70% of "Total Recharge Amount" to identify the recharge Amount Range for High value customer
print(df['Total_Recharge_Amount'].describe(percentiles = [0.7]))
print("\n70% of Total Recharge Amount of first 2 months are {}".format(df['Total_Recharge_Amount'].describe(percentiles = [0.7])[5]))

count    99999.000000
mean       650.477585
std        741.666932
min          0.000000
50%        458.000000
70%        737.000000
max      75525.000000
Name: Total_Recharge_Amount, dtype: float64

70% of Total Recharge Amount of first 2 months are 737.0


#### * Filter High Value Customer from main data frame

In [309]:
df = df[df['Total_Recharge_Amount'] > 737].reset_index(drop=True)
print("\nTotal High Value Customer Count ==> {}".format(df.shape[0]))
df.drop(columns=['Total_Recharge_Amount'], inplace=True)


Total High Value Customer Count ==> 29979


#### * Null Value Checking and Drop High Null Value Columns

In [310]:
#Get Null Percentage in dataFrame and Filter
nullPercentageLimit = 50
columns_More_Than_50_PercentNull = getColumnsBasedOnNullPercent(df,nullPercentageLimit)
#Drop Columns with More than 50% NUll
df = df.loc[:, ~df.columns.isin(columns_More_Than_50_PercentNull)]

print("\nColumn List Dropped with More than 50% of Null Value:==>\n {}\n".format(columns_More_Than_50_PercentNull))


Column List Dropped with More than 50% of Null Value:==>
 ['date_of_last_rech_data_6' 'date_of_last_rech_data_7'
 'date_of_last_rech_data_8' 'date_of_last_rech_data_9' 'total_rech_data_6'
 'total_rech_data_7' 'total_rech_data_8' 'total_rech_data_9'
 'max_rech_data_6' 'max_rech_data_7' 'max_rech_data_8' 'max_rech_data_9'
 'count_rech_2g_6' 'count_rech_2g_7' 'count_rech_2g_8' 'count_rech_2g_9'
 'count_rech_3g_6' 'count_rech_3g_7' 'count_rech_3g_8' 'count_rech_3g_9'
 'av_rech_amt_data_6' 'av_rech_amt_data_7' 'av_rech_amt_data_8'
 'av_rech_amt_data_9' 'arpu_3g_6' 'arpu_3g_7' 'arpu_3g_8' 'arpu_3g_9'
 'arpu_2g_6' 'arpu_2g_7' 'arpu_2g_8' 'arpu_2g_9' 'night_pck_user_6'
 'night_pck_user_7' 'night_pck_user_8' 'night_pck_user_9' 'fb_user_6'
 'fb_user_7' 'fb_user_8' 'fb_user_9']



#### * Check Categorical Variables and Single Record Variables

In [318]:
singleCategoryColumns = df.loc[:, np.array(df.apply(lambda x: x.nunique() == 1))].columns
for eachSingleCatgory in singleCategoryColumns:
    print("{}: {}".format(eachSingleCatgory, df[eachSingleCatgory].unique()))
print("\n<=== Drop Single Category Columns, Other than last_date_of_month_6/7/8/9, as it will be used for Derive Columns ===>\n")
singleCategoryColumns = [x for x in singleCategoryColumns if x not in list(['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'last_date_of_month_9'])]
singleCategoryColumns = np.array(singleCategoryColumns)
df = df.loc[:, ~df.columns.isin(singleCategoryColumns)]
df['last_date_of_month_7'] = df['last_date_of_month_7'].fillna('7/31/2014')
df['last_date_of_month_8'] = df['last_date_of_month_8'].fillna('8/31/2014')
df['last_date_of_month_9'] = df['last_date_of_month_9'].fillna('9/30/2014')

last_date_of_month_6: ['6/30/2014']
last_date_of_month_7: ['7/31/2014' nan]
last_date_of_month_8: ['8/31/2014' nan]
last_date_of_month_9: ['9/30/2014' nan]

<=== Drop Single Category Columns, Other than last_date_of_month_6/7/8/9, as it will be used for Derive Columns ===>



#### * Analyze Null Value for Less than 50%

In [312]:
columns_Less_Than_50_PercentNull = getColumnsBasedOnNullPercent(df,nullPercentageLimit, limitType='Lower')
df_temp = df.loc[:, columns_Less_Than_50_PercentNull]
round(df_temp.isnull().sum()/len(df_temp.index) * 100,2)

last_date_of_month_7    0.10
last_date_of_month_8    0.52
last_date_of_month_9    1.20
onnet_mou_6             1.05
onnet_mou_7             1.01
onnet_mou_8             3.13
onnet_mou_9             5.68
offnet_mou_6            1.05
offnet_mou_7            1.01
offnet_mou_8            3.13
offnet_mou_9            5.68
roam_ic_mou_6           1.05
roam_ic_mou_7           1.01
roam_ic_mou_8           3.13
roam_ic_mou_9           5.68
roam_og_mou_6           1.05
roam_og_mou_7           1.01
roam_og_mou_8           3.13
roam_og_mou_9           5.68
loc_og_t2t_mou_6        1.05
loc_og_t2t_mou_7        1.01
loc_og_t2t_mou_8        3.13
loc_og_t2t_mou_9        5.68
loc_og_t2m_mou_6        1.05
loc_og_t2m_mou_7        1.01
loc_og_t2m_mou_8        3.13
loc_og_t2m_mou_9        5.68
loc_og_t2f_mou_6        1.05
loc_og_t2f_mou_7        1.01
loc_og_t2f_mou_8        3.13
                        ... 
std_ic_t2t_mou_8        3.13
std_ic_t2t_mou_9        5.68
std_ic_t2m_mou_6        1.05
std_ic_t2m_mou

#### * As the Null % is very less, lets see if Null Value Can be imputed with some value

In [314]:
column_Month_6, column_Month_7, column_Month_8, column_Month_9, column_Common = returnColumnsByMonth(df_temp)

print("Month 6 Columns Count ==> {}".format(len(column_Month_6)))
print("Month 7 Columns Count ==> {}".format(len(column_Month_7)))
print("Month 8 Columns Count ==> {}".format(len(column_Month_8)))
print("Month 9 Columns Count ==> {}".format(len(column_Month_9)))
print("Common Columns Count ==> {}".format(len(column_Common)))
print("==> All Months are having same columns with less% of Null Value")
print(np.array(column_Month_7))
df_temp.loc[:, column_Month_7].head()

Month 6 Columns Count ==> 28
Month 7 Columns Count ==> 29
Month 8 Columns Count ==> 29
Month 9 Columns Count ==> 29
Common Columns Count ==> 0
==> All Months are having same columns with less% of Null Value
['last_date_of_month_7' 'onnet_mou_7' 'offnet_mou_7' 'roam_ic_mou_7'
 'roam_og_mou_7' 'loc_og_t2t_mou_7' 'loc_og_t2m_mou_7' 'loc_og_t2f_mou_7'
 'loc_og_t2c_mou_7' 'loc_og_mou_7' 'std_og_t2t_mou_7' 'std_og_t2m_mou_7'
 'std_og_t2f_mou_7' 'std_og_mou_7' 'isd_og_mou_7' 'spl_og_mou_7'
 'og_others_7' 'loc_ic_t2t_mou_7' 'loc_ic_t2m_mou_7' 'loc_ic_t2f_mou_7'
 'loc_ic_mou_7' 'std_ic_t2t_mou_7' 'std_ic_t2m_mou_7' 'std_ic_t2f_mou_7'
 'std_ic_mou_7' 'spl_ic_mou_7' 'isd_ic_mou_7' 'ic_others_7'
 'date_of_last_rech_7']


,last_date_of_month_7,onnet_mou_7,offnet_mou_7,roam_ic_mou_7,roam_og_mou_7,loc_og_t2t_mou_7,loc_og_t2m_mou_7,loc_og_t2f_mou_7,loc_og_t2c_mou_7,loc_og_mou_7,...,loc_ic_t2f_mou_7,loc_ic_mou_7,std_ic_t2t_mou_7,std_ic_t2m_mou_7,std_ic_t2f_mou_7,std_ic_mou_7,spl_ic_mou_7,isd_ic_mou_7,ic_others_7,date_of_last_rech_7
0,7/31/2014,54.68,567.16,33.49,12.59,31.38,447.38,55.14,0.0,533.91,...,101.46,355.23,11.83,126.99,34.24,173.08,0.0,14.53,15.19,7/25/2014
1,7/31/2014,351.03,80.63,0.00,0.00,217.59,70.58,0.00,0.0,288.18,...,0.00,23.83,0.58,4.08,0.00,4.66,0.0,0.00,0.00,7/31/2014
2,7/31/2014,108.39,119.28,144.24,35.26,6.19,47.28,0.00,0.0,53.48,...,0.00,26.11,38.29,29.79,0.00,68.09,0.0,62.11,16.24,7/22/2014
3,7/31/2014,74.01,229.74,2.83,17.74,65.16,145.99,4.48,0.0,215.64,...,8.68,229.83,78.64,73.08,0.00,151.73,0.0,0.00,0.00,7/31/2014
4,7/31/2014,9.28,0.00,4.76,8.46,0.00,0.00,0.00,0.0,0.00,...,0.00,2.31,0.00,0.00,0.00,0.00,0.0,0.00,0.00,7/30/2014


In [350]:
df['days_from_LastRechage_6'] = daysFromLastRechargeInMonth(df['date_of_last_rech_6'], df['last_date_of_month_6'])
df['days_from_LastRechage_7'] = daysFromLastRechargeInMonth(df['date_of_last_rech_7'], df['last_date_of_month_7'])
df['days_from_LastRechage_8'] = daysFromLastRechargeInMonth(df['date_of_last_rech_8'], df['last_date_of_month_8'])
df['days_from_LastRechage_9'] = daysFromLastRechargeInMonth(df['date_of_last_rech_9'], df['last_date_of_month_9'])

AttributeError: module 'datetime' has no attribute 'strptime'

In [343]:
df['days_from_LastRechage_7']

0        30
1        30
2        30
3        30
4        30
5        30
6        30
7        30
8        30
9        30
10       30
11       30
12       30
13       30
14       30
15       30
16       30
17       30
18       30
19       30
20       30
21       30
22       30
23       30
24       30
25       30
26       30
27       30
28       30
29       30
         ..
99969    30
99970    30
99971    30
99972    30
99973    30
99974    30
99975    30
99976    30
99977    30
99978    30
99979    30
99980    30
99981    30
99982    30
99983    30
99984    30
99985    30
99986    30
99987    30
99988    30
99989    30
99990    30
99991    30
99992    30
99993    30
99994    30
99995    30
99996    30
99997    30
99998    30
Name: days_from_LastRechage_7, Length: 99999, dtype: int64